In [25]:
from binance.client import Client
import pandas as pd
import numpy as np
import math
import time
import statistics
from coin_data import data
from config import api_key, api_secret

In [26]:
print(data)

{'ROSEUSDT': {'i1': 0.001, 'i2': 0.0001}, 'MANAUSDT': {'i1': 0.1, 'i2': 0.01, 'sl': 0.7}, 'ADAUSDT': {'i1': 0.01, 'i2': 0.001}, 'BTCUSDT': {'i1': 100, 'i2': 50}, 'MATICUSDT': {'i1': 0.01, 'i2': 0.001}}


In [27]:
client = Client(api_key=api_key, api_secret= api_secret)
ticker = 'MANAUSDT'

# 1. Extraer datos de la moneda

- Traemos toda la info de todas las monedas de futuros de binance

In [28]:
info = client.futures_exchange_info()

- Buscamos la moneda que necesitamos
- Y extraemos la información que creamos necesaria
- Esto lo deberíamos ejecutar 1 sola vez para todas las monedas y guardarlo para solo consultarlo como diccionario y no extraer la información cada vez
- Aunque los datos traen la precisión para el precio, en algunas monedas está mal. Por lo tanto, se crea la función que retorna la cantidad de decimales a través de otra propiedad listada

In [29]:
def get_precision(texto:str) -> int:
    for i in range(2, len(texto)):
        if texto[i] == '1':
            return i-1

In [30]:
for d in info['symbols']:
    if d['pair']==ticker:
        print(d)
        print("")
        print(f"Moneda: {d['pair']}\nSubtipo: {d['underlyingSubType']}\nPrecisión Precio: {get_precision(d['filters'][0]['tickSize'])} decimales")


# 2. Obtener Shock Points

# a. Libro de ordenes

- Vamos a obtener el libro de órdenes de la moneda
- Luego separamos las ventas de las compras y las colocamos en un dataframe para mejor organización
- Como nota, lo que trae el libro no es la cantidad total de monedas por bloque sino el total de USDT
- En el caso de las ventas, para mostrarlo como lo vemos en Binance (mayor a menor) ordenamis descebdentemente

In [31]:
order_book = client.futures_order_book(symbol=ticker, limit=1000)
#order_book

BinanceAPIException: APIError(code=-1121): Invalid symbol.

In [ ]:
ventas = order_book['asks']
ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)
ventas

/tmp/ipykernel_27299/4237633481.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only.
  ventas = pd.DataFrame(ventas, columns=['precio','total_usdt']).sort_index(0, ascending=False)


,precio,total_usdt
999,1.02500,5736
998,1.02490,70
997,1.02480,302
996,1.02470,76
995,1.02460,36
...,...,...
4,0.92370,32600
3,0.92360,30614
2,0.92350,21278
1,0.92340,9091


In [ ]:
compras = order_book['bids']
compras = pd.DataFrame(compras, columns=['precio','total_usdt'])
compras

,precio,total_usdt
0,0.92320,3552
1,0.92310,11481
2,0.92300,22075
3,0.92290,27528
4,0.92280,39258
...,...,...
995,0.82350,4056
996,0.82340,36
997,0.82330,7
998,0.82320,1443


## b. Obtención de rangos de precios

- Para obtener los rangos y obtener los puntos a cómo lo hace el gafas, debemos crear rangos
- Para crear los rangos necesitamos el precio mínimo y máximo para redondearlos hacia abajo y arriba respecto a los múltiplos de los libros

### i. Obtención de máximos y mínimos precios

- Para poder hacer esto, tenemos que decirle al Dataframe qué tipo de valores son las columnas
- Esto es porque, como se puede observar a continuación, los marca como object

In [ ]:
compras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   precio      1000 non-null   object
 1   total_usdt  1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


- Para modificar el tipo de objetos de las columnas, hacemos:

In [ ]:
ventas[['precio','total_usdt']] = ventas[['precio','total_usdt']].astype('float')
ventas

,precio,total_usdt
999,1.0250,5736.0
998,1.0249,70.0
997,1.0248,302.0
996,1.0247,76.0
995,1.0246,36.0
...,...,...
4,0.9237,32600.0
3,0.9236,30614.0
2,0.9235,21278.0
1,0.9234,9091.0


In [ ]:
compras[['precio','total_usdt']] = compras[['precio','total_usdt']].astype('float')
compras

,precio,total_usdt
0,0.9232,3552.0
1,0.9231,11481.0
2,0.9230,22075.0
3,0.9229,27528.0
4,0.9228,39258.0
...,...,...
995,0.8235,4056.0
996,0.8234,36.0
997,0.8233,7.0
998,0.8232,1443.0


- Ahora sí, obtenemos los máximos y mínimos

In [ ]:
max_ventas = ventas['precio'].max()
min_ventas = ventas['precio'].min()
print('Ventas(mín-máx): {} - {}'.format(min_ventas, max_ventas))

max_compras = compras['precio'].max()
min_compras = compras['precio'].min()
print('Compras(mín-máx): {} - {}'.format(min_compras, max_compras))

Ventas(mín-máx): 0.9233 - 1.025
Compras(mín-máx): 0.8231 - 0.9232


### ii. Obtención de Dataframes con los Rangos

- Para esto, necesitamos tener los múltiplos de las moendas cómo lo hace el gafas
- Desafortunadamante, aún no encuentro cómo obtener los múltiplos de binance por lo que toca escribirlos en el archivo coin_data.py
- Para obtener el inicio de inicio y final de un rango, hacemos:

1. (precio_minimo / multiplo) ; donde el múltiplo es el que obtenemos de coind_data en i1
2. Luego redondeamos al entero inferior con math.floor()
3. Y multiplicamos por el múltiplo nuevamente para obtener el inicio del rango
4. (precio_maximo / multiplo)
5. redondeamos hacia el entero superior con math.ceil()
6. Y multiplicamos por el múltiplo nuevamente para obtener el máximo del rango

#### Obtención de valores iniciales y finales de rango con primer múltiplo

In [ ]:
# Ventas
inicio_rango_ventas_1 = ( math.floor( (min_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_ventas_1 = ( math.ceil( (max_ventas / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_1,final_rango_ventas_1))
# Compras
inicio_rango_compras_1 = ( math.floor( (min_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
final_rango_compras_1 = ( math.ceil( (max_compras / data[ticker]['i1']) ) ) * data[ticker]['i1']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_1,final_rango_compras_1))

Rango Ventas: 0.92 - 1.03
Rango Compras: 0.8200000000000001 - 0.93


#### Obtención de valores iniciales y finales de rango con segundo múltiplo

In [ ]:
# Ventas
inicio_rango_ventas_2 = ( math.floor( (min_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_ventas_2 = ( math.ceil( (max_ventas / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Ventas: {} - {}'.format(inicio_rango_ventas_2,final_rango_ventas_2))
# Compras
inicio_rango_compras_2 = ( math.floor( (min_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
final_rango_compras_2 = ( math.ceil( (max_compras / data[ticker]['i2']) ) ) * data[ticker]['i2']
print('Rango Compras: {} - {}'.format(inicio_rango_compras_2,final_rango_compras_2))

Rango Ventas: 0.923 - 1.025
Rango Compras: 0.8230000000000001 - 0.924


### iii. Obtención de los dataframes agrupados por rangos

- Ya que tenemos los valores inicial y final de los rangos, creamos el Dataframe con datos agrupados por rangos
- Solo que, antes tenemos que tener un rango con incrementos de acuerdo al múltiplo. Para esto usamos np.arange()

- Con múltiplo 1

In [ ]:
rango_v_1 = np.arange(inicio_rango_ventas_1, final_rango_ventas_1+data[ticker]['i1'], data[ticker]['i1'])
ventas_ag_1 = ventas.groupby(pd.cut(ventas.precio,rango_v_1)).sum()
print('Ventas agrupadas segun incremento 1')
print(rango_v_1)
print(ventas_ag_1.sort_index(ascending=False))
rango_c_1 = np.arange(inicio_rango_compras_1, final_rango_compras_1+data[ticker]['i1'], data[ticker]['i1'])
compras_ag_1 = compras.groupby(pd.cut(compras.precio,rango_c_1)).sum()
print('\nCompras agrupadas segun incremento 1')
print(rango_c_1)
print(compras_ag_1.sort_index())

Ventas agrupadas segun incremento 1
[0.92 0.93 0.94 0.95 0.96 0.97 0.98 0.99 1.   1.01 1.02 1.03]
               precio  total_usdt
precio                           
(1.02, 1.03]  48.0583    604290.0
(1.01, 1.02]  97.4552    510768.0
(1.0, 1.01]   95.4902    156131.0
(0.99, 1.0]   97.5160    252304.0
(0.98, 0.99]  96.5334    336419.0
(0.97, 0.98]  97.5050    362090.0
(0.96, 0.97]  95.5369    470162.0
(0.95, 0.96]  95.5050    781734.0
(0.94, 0.95]  93.5643    978134.0
(0.93, 0.94]  93.5050   1979200.0
(0.92, 0.93]  63.0122   3160526.0

Compras agrupadas segun incremento 1
[0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93]
               precio  total_usdt
precio                           
(0.82, 0.83]  57.8585    200652.0
(0.83, 0.84]  82.6666    312272.0
(0.84, 0.85]  84.5050    480805.0
(0.85, 0.86]  85.5050    473934.0
(0.86, 0.87]  85.6423    397905.0
(0.87, 0.88]  87.5050    548790.0
(0.88, 0.89]  88.5050    612144.0
(0.89, 0.9]   89.5050   1589840.0
(0.9, 0.91]   90.505

- con múltiplo 2

In [ ]:
rango_v_2 = np.arange(inicio_rango_ventas_2, final_rango_ventas_2+data[ticker]['i2'], data[ticker]['i2'])
ventas_ag_2 = ventas.groupby(pd.cut(ventas.precio,rango_v_2)).sum()
print('Ventas agrupadas segun incremento 2')
#print(rango_v_2)
print(ventas_ag_2.sort_index(ascending=False))
rango_c_2 = np.arange(inicio_rango_compras_2, final_rango_compras_2+data[ticker]['i2'], data[ticker]['i2'])
compras_ag_2 = compras.groupby(pd.cut(compras.precio,rango_c_2)).sum()
print('\nCompras agrupadas segun incremento 2')
#print(rango_c_2)
print(compras_ag_2.sort_index(ascending=False))

Ventas agrupadas segun incremento 2
                 precio  total_usdt
precio                             
(1.024, 1.025]   9.2212     15874.0
(1.023, 1.024]   9.2121      6590.0
(1.022, 1.023]  10.2255     76730.0
(1.021, 1.022]   9.1940      7402.0
(1.02, 1.021]   10.2055    497694.0
...                 ...         ...
(0.927, 0.928]   9.2755    672070.0
(0.926, 0.927]   9.2655    644421.0
(0.925, 0.926]   9.2555    225893.0
(0.924, 0.925]   9.2455    511305.0
(0.923, 0.924]   7.3892    207694.0

[102 rows x 2 columns]

Compras agrupadas segun incremento 2
                precio  total_usdt
precio                            
(0.923, 0.924]  1.8463     15033.0
(0.922, 0.923]  9.2255    334357.0
(0.921, 0.922]  9.2155    259269.0
(0.92, 0.921]   9.2055    307669.0
(0.919, 0.92]   9.1955    538471.0
...                ...         ...
(0.827, 0.828]  8.2755     44084.0
(0.826, 0.827]  8.2655     36431.0
(0.825, 0.826]  8.2555     13824.0
(0.824, 0.825]  8.2455     11326.0
(0.823, 0.824]

#### Obtención de los shock points

- Ahora, para obtener los shock point, necesitamos obtener el dato donde el total usdt es máximo
- Traer el índice y extraer:
    - Para ventas: el lado derecho del intervalo. Porque es la parte superior del intervalo
    - Para compras: el lado izquiero del intervalo. Porque es la parte inferior del intervalo

In [ ]:
from numpy import sort


sp_v_1 = ventas_ag_1['total_usdt'].idxmax().right
#print(sp_v_1)
#print(ventas_ag_2)
#print(ventas_ag_2['total_usdt'].max())
#print(ventas_ag_2['total_usdt'].idxmax())
#print(ventas_ag_2['total_usdt'].idxmax())
sp_v_2 = ventas_ag_2['total_usdt'].idxmax().right
sp_v = []
sp_v.append(sp_v_1)
sp_v.append(sp_v_2)
sp_v = sort(sp_v)
sp_v_1 = sp_v[1]
sp_v_2 = sp_v[0]
print(f'Ventas:\n***************\n2.  {sp_v_1}\n1.  {sp_v_2}')

Ventas:
***************
2.  0.93
1.  0.928


In [ ]:
sp_c_1 = compras_ag_1['total_usdt'].idxmax().left
#print(sp_c_1)
#print(compras_ag_2)
#print(compras_ag_2['total_usdt'].max())
#print(compras_ag_2['total_usdt'].idxmax())
sp_c_2 = compras_ag_2['total_usdt'].idxmax().left
#print(sp_c_2)
sp_c = []
sp_c.append(sp_c_1)
sp_c.append(sp_c_2)
sp_c = sort(sp_c)
sp_c_1 = sp_c[1]
sp_c_2 = sp_c[0]
print(f'Compras:\n***************\n1.  {sp_c_1}\n2.  {sp_c_2}')

Compras:
***************
1.  0.914
2.  0.91


# 3. Analizar Shock Points

- El análisis de los puntos se hará de la siguiente manera:
    - Para ventas, obtenemos la distancia o porcentaje entre puntos de venta: pct_v = ( (venta_fin - venta_inicio)/venta_din ) * 100
    - Para compras, lo calculasmo así: pct_c = ( (compra_inicio - compra_fin)/compra_inicio ) * 100
- Luego, debemos obtener la distancia entre el primer punto de venta y de compras, en este caso tenemos una ligera variación por lo que obtendremos el promedio. 
    - (1)=  (compras_inicio - ventas_inicio)/compras_inicio ) * -100
    - (2)=  (ventas_inicio - compras_inicio)/ventas_inicio ) * 100
    - pct_sep = promedio(1,2)

In [ ]:
pct_ventas = round( ( (sp_v_1 - sp_v_2) / sp_v_2 ) * 100, 2 )
print(f'Distancia entre ventas: {pct_ventas}%')

pct_compras = round( ( (sp_c_1 - sp_c_2) / sp_c_1 ) * 100, 2)
print(f'Distancia entre compraas: {pct_compras}%')

pct_sep_1 = (( (sp_c_1 - sp_v_2) / sp_c_1 ) * -100)
pct_sep_2 = ((sp_v_2 - sp_c_1) / sp_v_2 ) * 100
pct_sep = round((pct_sep_1+pct_sep_2)/2,2)
print(f'Porcentaje entre compra y venta: {pct_sep}%')

Distancia entre ventas: 0.22%
Distancia entre compraas: 0.44%
Porcentaje entre compra y venta: 1.52%


# 4. Resultado del Análisis de los Shock Points

- Una vez que tenemos las distancias, procedemos a comparar lo siguiente:
    - Para compras:
        - Distancia entre compra y venta debe ser mayor a la distancia entre compras por 2.5 o más veces
    - Para ventas:
        - - Distancia entre compra y venta debe ser mayor a la distancia entre ventas por 2.5 o más veces
- Si se cumplen esas condiciones enviamos señal para compra y/o venta
- Una nota aquí es que, a la distancia entre ventas o entre compras, habría que aumentarle la distancia entre el segundo shock_point y el SL

In [ ]:
ratio = 2.5
flag_compra = False
flag_venta = False
if (pct_sep/pct_ventas) >= ratio: # Aquí es pct_sep/(pct_ventas+SL)
    print(f'Se puede vender en {ticker}')
    flag_venta = True
if (pct_sep/pct_compras) >= ratio: # Aquí es pct_sep/(pct_compras+SL)
    print(f'Se puede comprar en {ticker}')
    flag_compra = True
if not flag_compra and not flag_venta:
    print('Nada que hacer, sigue buscando')

Se puede vender en MATICUSDT
Se puede comprar en MATICUSDT


# 5. Calcular SL, TP y Recompras

- En el caso que sí se habilite alguna operación debemos tener una gestión por moneda dentro del archivo coin_data para determinar qué tanta separación entre el segundo shock point y nuestro SL debe haber
- Esto lo veremos de acuerdo a nuestro análisis de nuestras monedas teniendo uno por default o uno por moneda

## Entrada

In [ ]:
if flag_venta:
    print(f'Entrada Venta:\t${sp_v_2}')

if flag_compra:
    print(f'Entrada Compra:\t${sp_c_1}')

Entrada Venta:	$0.928
Entrada Compra:	$0.914


## SL

- Primero debemos traer el balance de nuestra cuenta de futuros

In [ ]:
import os

In [ ]:
#x = 0
# while True:
#    if x < 100:
#        actual_price = client.futures_mark_price(symbol=ticker)
#        os.system('clear')
#        print('Precio actual {}'.format(actual_price))
#        x += 1


## Recompras / Reventas

- Para las recompras usaremos la misma configuración del gafas:
    - Compras a cada 2%
    - Comprando el 40% de las monedas que esten al momento
    - SL del 10% del capital

# 6. Calcular recompras

# 7. Ejecutar órdenes